In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

In [2]:
conf = (
    pyspark.SparkConf()
        .setAppName('iceberg_demo')
  		#SQL Extensions
        #Configuring Catalog
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
        .set('spark.sql.catalog.iceberg_demo', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.iceberg_demo.type', 'rest')
        .set('spark.sql.catalog.iceberg_demo.uri', 'http://rest:8181')
        .set('spark.sql.catalog.iceberg_demo.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        .set('spark.sql.catalog.iceberg_demo.warehouse', 's3://warehouse/wh/')
        .set('spark.sql.catalog.iceberg_demo.s3.endpoint', 'http://minio:9000')
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

Spark Running


24/08/11 08:19:37 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
## Creating table
from pyspark.sql.types import DoubleType, FloatType, LongType, StructType,StructField, StringType
schema = StructType([
  StructField("vendor_id", LongType(), True),
  StructField("trip_id", LongType(), True),
  StructField("trip_distance", FloatType(), True),
  StructField("fare_amount", DoubleType(), True),
  StructField("store_and_fwd_flag", StringType(), True)
])

df = spark.createDataFrame([], schema)
df.writeTo("iceberg_demo.nyc.taxis").create()

In [6]:
## Inserting data into table
schema = spark.table("iceberg_demo.nyc.taxis").schema
data = [
    (1, 1000371, 1.8, 15.32, "N"),
    (2, 1000372, 2.5, 22.15, "N"),
    (2, 1000373, 0.9, 9.01, "N"),
    (1, 1000374, 8.4, 42.13, "Y")
  ]
df = spark.createDataFrame(data, schema)
df.writeTo("iceberg_demo.nyc.taxis").append()

In [7]:
csv_df = spark.read.format("csv").option("header", "true").load("file:///home/iceberg/datasets/df_open_2023.csv")

In [31]:
csv_df.writeTo("iceberg_demo.nyc.df_open_2023").create()

In [34]:
spark.sql("SELECT * FROM iceberg_demo.nyc.df_open_2023 limit 10;").show()

+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|   competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|      469656|    Jeffrey Adler|  Jeffrey|      Adler|   ACT|     M|                 CA|             Canada|      35|North America East|      18059| CrossFit Wonderland| 29| 69 in|197 lb|          1|         107|       1|2023|
|      300638|   Tola Morakinyo|     Tola|  Morakinyo|   ACT|     M|                 US|    